In [1]:
def is_inside(points, target):
    num_intersects = 0
    furthest_x = max(points)[0] + 1
    right_point = (furthest_x, target[1])
    points.append(points[0])
    for i in range(0, len(points) - 1):
        point1 = points[i]
        point2 = points[i+1]
        if do_intersect(point1, point2, target, right_point):
            num_intersects += 1
            if located_on_line(point1, point2, target):
                return True
    return num_intersects % 2 != 0

def do_intersect(pointA, pointB, pointC, pointD):
    a1 = pointB[1] - pointA[1]
    b1 = pointA[0] - pointB[0]
    c1 = a1 * (pointA[0]) + b1 * (pointA[1])
    a2 = pointD[1] - pointC[1]
    b2 = pointC[0] - pointD[0]
    c2 = a2 * (pointC[0]) + b2 * (pointC[1])
    determinant = a1 * b2 - a2 * b1
    if (determinant == 0):
        return False
    else:
        x = (c1*b2 - c2*b1)/determinant
        y = (a1*c2 - a2*c1)/determinant
        isValidX = min(pointA[0], pointB[0]) <= x <= max(pointA[0], pointB[0])
        isValidY = min(pointA[1], pointB[1]) <= y <= max(pointA[1], pointB[1])
        isToRight = x >= pointC[0]

        return isValidX and isValidY and isToRight


def located_on_line(pointA, pointB, pointC):
    """
    Given 3 collinear points p, q, and r, check if point q is on the line of 'pr'
    Source: https://www.geeksforgeeks.org/how-to-check-if-a-given-point-lies-inside-a-polygon/#:~:text=1)%20Draw%20a%20horizontal%20line,on%20an%20edge%20of%20polygon.
    """
    if ((pointC[0] <= max(pointA[0], pointB[0]))
            & (pointC[0] >= min(pointA[0], pointB[0]))
            & (pointC[1] <= max(pointA[1], pointB[1]))
            & (pointC[1] >= min(pointA[1], pointB[1]))):
        return True
    return False

In [2]:
polygon1 = [(0, 0), (10, 0), (10, 10), (0, 10)]

p1 = (20, 20)  # false
p2 = (5, 5)  # true
p3 = (-23, -23)  # false
p4 = (0, 3)  # true

polygon2 = [(0, 0), (5, 0), (5, 5), (3, 3)]

p5 = (3, 3)  # true
p6 = (5, 1)  # true
p7 = (8, 1)  # false
p8 = (2.4, 0) # true

In [3]:
print(is_inside(polygon1, p1))
print(is_inside(polygon1, p2))
print(is_inside(polygon1, p3))
print(is_inside(polygon1, p4))
print(is_inside(polygon2, p5))
print(is_inside(polygon2, p6))
print(is_inside(polygon2, p7))
print(is_inside(polygon2, p8))

False
True
False
True
True
True
False
True


In [6]:
import math, random
from typing import List, Tuple


def generate_polygon(center: Tuple[float, float], avg_radius: float,
                     irregularity: float, spikiness: float,
                     num_vertices: int) -> List[Tuple[float, float]]:
    """
    Start with the center of the polygon at center, then creates the
    polygon by sampling points on a circle around the center.
    Random noise is added by varying the angular spacing between
    sequential points, and by varying the radial distance of each
    point from the centre.

    Args:
        center (Tuple[float, float]):
            a pair representing the center of the circumference used
            to generate the polygon.
        avg_radius (float):
            the average radius (distance of each generated vertex to
            the center of the circumference) used to generate points
            with a normal distribution.
        irregularity (float):
            variance of the spacing of the angles between consecutive
            vertices.
        spikiness (float):
            variance of the distance of each vertex to the center of
            the circumference.
        num_vertices (int):
            the number of vertices of the polygon.
    Returns:
        List[Tuple[float, float]]: list of vertices, in CCW order.
    """
    # Parameter check
    if irregularity < 0 or irregularity > 1:
        raise ValueError("Irregularity must be between 0 and 1.")
    if spikiness < 0 or spikiness > 1:
        raise ValueError("Spikiness must be between 0 and 1.")

    irregularity *= 2 * math.pi / num_vertices
    spikiness *= avg_radius
    angle_steps = random_angle_steps(num_vertices, irregularity)

    # now generate the points
    points = []
    angle = random.uniform(0, 2 * math.pi)
    for i in range(num_vertices):
        radius = clip(random.gauss(avg_radius, spikiness), 0, 2 * avg_radius)
        point = (center[0] + radius * math.cos(angle),
                 center[1] + radius * math.sin(angle))
        points.append(point)
        angle += angle_steps[i]

    return points

def random_angle_steps(steps: int, irregularity: float) -> List[float]:
    """Generates the division of a circumference in random angles.

    Args:
        steps (int):
            the number of angles to generate.
        irregularity (float):
            variance of the spacing of the angles between consecutive vertices.
    Returns:
        List[float]: the list of the random angles.
    """
    # generate n angle steps
    angles = []
    lower = (2 * math.pi / steps) - irregularity
    upper = (2 * math.pi / steps) + irregularity
    cumsum = 0
    for i in range(steps):
        angle = random.uniform(lower, upper)
        angles.append(angle)
        cumsum += angle

    # normalize the steps so that point 0 and point n+1 are the same
    cumsum /= (2 * math.pi)
    for i in range(steps):
        angles[i] /= cumsum
    return angles

def clip(value, lower, upper):
    """
    Given an interval, values outside the interval are clipped to the interval
    edges.
    """
    return min(upper, max(value, lower))


vertices = generate_polygon(center=(250, 250),
                            avg_radius=100,
                            irregularity=0.35,
                            spikiness=0.2,
                            num_vertices=16)

In [8]:
vertices

[(233.40165668738183, 351.55247146315185),
 (187.43234372496468, 330.21608098145276),
 (151.2971637088135, 310.99031494411287),
 (157.47366862422425, 271.8074260494148),
 (137.78581177724783, 222.90370556237193),
 (184.45010641188875, 192.34896103138385),
 (198.6154327741612, 159.48686625761644),
 (225.53957402447801, 143.7279633915656),
 (270.3108817514015, 170.99073907959655),
 (333.0370727599876, 146.82034764035393),
 (335.45983240551396, 208.13416244088282),
 (325.5445804195719, 239.9325438417781),
 (326.5045158711066, 262.74877354593724),
 (342.18716409973365, 297.1622862875041),
 (306.27387729460077, 323.3374589152179),
 (286.7001301875804, 374.0633875626795)]